In [2]:
import requests
import json
from configuration import api_key
import pandas as pd
from pathlib import Path



park_visitors_csv_file = Path(../Resources/Annual Park Ranking Report (1979 - Last Calendar Year).csv)



sAXQouXWmVhAS1Cspb397Lszyf2nhBFG6uCRB664


In [16]:
!pip install requests